In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv("Ecommerce_Customers.csv", 
                      inferSchema=True,
                     header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [69]:
# You have a list of input cols, features, and output,single feature.
assembler = VectorAssembler(inputCols=['Avg Session Length',
             'Time on App',
             'Time on Website',
             'Length of Membership'],
              outputCol="features")

In [70]:
#Add new output col based of features, named features
output = assembler.transform(data)

In [71]:
# New feature col
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [72]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [73]:
final_data = output.select("features", "Yearly Amount Spent")

In [74]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [75]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [76]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                335|
|   mean|  497.8141133372085|
| stddev|  77.91916150423486|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [77]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                165|
|   mean| 502.35934037169966|
| stddev|  82.23243439236747|
|    min|  282.4712457199145|
|    max|  689.2356997616951|
+-------+-------------------+



In [78]:
# features col is feature named
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [79]:
lr_model = lr.fit(train_data)

21/10/29 11:26:04 WARN Instrumentation: [434816f9] regParam is zero, which might cause numerical instability and overfitting.


In [80]:
test_results = lr_model.evaluate(test_data)

In [81]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.947460129271349|
| -3.817903316529339|
|-12.734236664408343|
|-21.763039247637494|
| -7.590073546055919|
| 11.083180641837771|
|  4.318897546773883|
| -7.081699989298045|
|  4.887854672992773|
|-2.0911816402709746|
| 17.711563777299773|
|-11.703877166373786|
|-17.395281054129157|
|-2.0655308123079408|
|  1.949668249678325|
|-13.158089029415805|
|   6.49032636897914|
| -8.310397965399716|
|  4.093728766897982|
| 18.812269240540218|
+-------------------+
only showing top 20 rows



In [82]:
test_results.rootMeanSquaredError

10.12216460243034

In [83]:
test_results.r2

0.9847559420021799

In [84]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [85]:
unlabeled_data = test_data.select('features')

In [87]:
predictions = lr_model.transform(unlabeled_data)

In [88]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.69289094335613|
|[30.4925366965402...|286.28914903644386|
|[31.0662181616375...| 461.6675298720827|
|[31.1239743499119...|508.71009308740327|
|[31.1280900496166...| 564.8427602931106|
|[31.1695067987115...|416.27335016045504|
|[31.3662121671876...|426.26998500971104|
|[31.4474464941278...|425.68444208452206|
|[31.5316044825729...| 431.6277510563698|
|[31.5761319713222...| 543.3177656295993|
|[31.6098395733896...| 426.8339858738084|
|[31.8093003166791...|  548.475776529215|
|[31.8164283341993...| 518.5177725577855|
|[31.8186165667690...| 448.4842041824436|
|[31.8293464559211...|383.20266973829666|
|[31.9365486184489...|  440.357473924744|
|[31.9764800614612...|324.10411966512106|
|[32.0085045178551...| 451.5076189941551|
|[32.0123007682454...|488.85132429906025|
|[32.0180740106320...| 338.9708415047751|
+--------------------+------------

In [89]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.12216460243034
MSE: 102.45821623869375
